In [1]:
import tensorflow as tf
import keras as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def create_model(layers = [64,64,128,128]):
    model = Sequential()
    model.add(Conv2D(layers[0], (3, 3), input_shape = (8, 8, 2)))
    for layer in layers[1:]:
        model.add(Conv2D(layer, (3, 3)))
    model.add(Dense(units = 128))
    model.add(Dense(units = 60, activation = 'sigmoid'))
    return model